## Resumen de la notebook

Esta notebook contiene las funciones necesarias para obtener, a partir de la database, diferentes features que forman parte de la database pero pueden ser calculadas a partir de los datos que se tienen

In [1]:
import numpy as np
import pandas as pd
import datetime
from datetime import datetime, time
import scipy.io as sio

In [2]:
def tiempo_en_segundos(df):
    '''Da el tiempo en segundos que marco el acelerómetro
        usando la columna Time y la agrega al final de la database'''
    t_en_segundos = []
    for i in range(len(df['Time'])):
        fecha_y_hora = datetime.strptime(df['Time'][i],'%H:%M:%S.%f')
        hora_en_segundos = fecha_y_hora.hour * 3600 + fecha_y_hora.minute * 60 + fecha_y_hora.second + fecha_y_hora.microsecond / 1e6
        t_en_segundos.append(hora_en_segundos)

    df['time_in_seconds'] = t_en_segundos

In [3]:
def tiempo_en_video(df):
    '''Da el tiempo en segundos que marco el video
        usando la columna Time y la agrega al final de la database'''
    video_segundos = []
    for i in range(len(df['Time'])):
        fecha_y_hora = datetime.strptime(df['Time'][i],'%H:%M:%S.%f')
        hora_en_segundos = fecha_y_hora.hour * 3600 + fecha_y_hora.minute * 60 + fecha_y_hora.second + fecha_y_hora.microsecond / 1e6 - 33390 + 85.4
        video_segundos.append(hora_en_segundos)

    df['Video_seconds'] = video_segundos

In [4]:
def estatica(df):
    '''Calcula la aceleración estática en cada una de las coordenadas
        usando el correspondiente DBA y la información del acelerómetro
        de esa coordenada. Agrega las columnas al fina de la database'''
    
    estatica_X = []
    for i in range(len(df['DBA-X'])):
        aux = df["Axis-X"][i]-df['DBA-X'][i]
        estatica_X.append(aux)
    
    estatica_Y = []
    for i in range(len(df['DBA-Y'])):
        aux = df["Axis-Y"][i]-df['DBA-Y'][i]
        estatica_Y.append(aux)
    
    estatica_Z = []
    for i in range(len(df['DBA-Z'])):
        aux = df["Axis-Z"][i]-df['DBA-Z'][i]
        estatica_Z.append(aux)
    
    df['Estatica-X'] = estatica_X
    df['Estatica-Y'] = estatica_Y
    df['Estatica-Z'] = estatica_Z

In [5]:
def odba_vedba(df):
    '''Calcula la aceleración dinámica sobre todas las coordenadas y la 
        aceleración estática vectorial usando la información de los DBA
        Agrega las columnas al fina de la database'''
    OBDA = []
    VeDBA = []
    for i in range(len(df['DBA-X'])):
        aux = df['DBA-X'][i] + df['DBA-Y'][i] +df['DBA-Z'][i]
        aux2 = np.sqrt(df['DBA-X'][i]**2 + df['DBA-Y'][i]**2 + df['DBA-Z'][i]**2)
        OBDA.append(aux)
        VeDBA.append(aux2)
    
    df['ODBA'] = OBDA
    df['VeDBA'] = VeDBA

In [6]:
def agregar_comp_no_medido(df, comportamiento):
    ''' Agregará una columna con header el comportamiento
        que se quiere registrar, a la derecha de los comportamientos
        ya medidos, en nuestro caso son 5 comportamientos, y la
        columna se comportará con NaNs ya que no fue medido'''
    df.insert(5,comportamiento, np.nan, allow_duplicates=False)

In [7]:
def a_comport(df, lista):
    '''agregará todos los comportamientos no medidos
        de la lista a la database'''
    for c in lista:
        agregar_comp_no_medido(df,c)

In [8]:
def completar_database(df, etiquetas,c_faltantes):
    tiempo_en_segundos(df)
    tiempo_en_video(df)
    estatica(df)
    odba_vedba(df)
    a_comport(etiquetas, c_faltantes)
    combined_df = pd.concat([df, etiquetas], axis=1)
    return combined_df

## Carga de datos 

Realizado con la tanda 1 a modo de ejemplo

In [9]:
df=pd.read_csv("T01_sincomportamiento.csv")
df.drop("Unnamed: 0", axis=1, inplace=True)
df.head()

,Unnamed: 0.1,Time,Axis-X,Axis-Y,Axis-Z,DBA-X,DBA-Y,DBA-Z,Pow-X,Pow-Y,Pow-Z
0,865,09:17:04.600,-0.062,-0.125,1.031,-0.05116,-0.125,1.02624,2.737028,1.718263,14.134036
1,866,09:17:04.640,-0.062,-0.125,1.016,-0.05056,-0.125,1.02624,2.727309,1.263568,12.277164
2,867,09:17:04.680,-0.062,-0.125,1.016,-0.05056,-0.125,1.02684,3.548803,1.499625,12.032834
3,868,09:17:04.720,-0.047,-0.125,1.031,-0.05056,-0.125,1.02744,2.810269,1.489273,14.679820
4,869,09:17:04.760,-0.062,-0.125,1.016,-0.04992,-0.125,1.02744,2.690873,2.037897,12.334172


In [10]:
etiquetas=pd.read_csv("T01_comportamientos.csv")
etiquetas.drop("Unnamed: 0", axis=1, inplace=True)
etiquetas.head()

,Otros,Reproductivo,Sacudida,Higiene,Baño de arena
0,1.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0


In [11]:
c_faltantes = ["Quietud", "Reposo Sentado","Vigilante Sentado",
               "Vigilante Parado", "Sacudida Sentado","Exploracion",
               "Pica el Piso","Picoteo Sentado","Acicalamiento",
               "Acicalamiento Erguido","Rascado con Pata",
               "Establización con Alas","Caminata","Camina Agachado",
               "Corre","Agarre","Monta","Contacto Cloacal"]
final = completar_database(df, etiquetas,c_faltantes)
final.head()

,Unnamed: 0.1,Time,Axis-X,Axis-Y,Axis-Z,DBA-X,DBA-Y,DBA-Z,Pow-X,Pow-Y,...,Acicalamiento Erguido,Acicalamiento,Picoteo Sentado,Pica el Piso,Exploracion,Sacudida Sentado,Vigilante Parado,Vigilante Sentado,Reposo Sentado,Quietud
0,865,09:17:04.600,-0.062,-0.125,1.031,-0.05116,-0.125,1.02624,2.737028,1.718263,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,866,09:17:04.640,-0.062,-0.125,1.016,-0.05056,-0.125,1.02624,2.727309,1.263568,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,867,09:17:04.680,-0.062,-0.125,1.016,-0.05056,-0.125,1.02684,3.548803,1.499625,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,868,09:17:04.720,-0.047,-0.125,1.031,-0.05056,-0.125,1.02744,2.810269,1.489273,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,869,09:17:04.760,-0.062,-0.125,1.016,-0.04992,-0.125,1.02744,2.690873,2.037897,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 597001 entries, 0 to 597000
Data columns (total 41 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   Unnamed: 0.1            597001 non-null  int64  
 1   Time                    597001 non-null  object 
 2   Axis-X                  597001 non-null  float64
 3   Axis-Y                  597001 non-null  float64
 4   Axis-Z                  597001 non-null  float64
 5   DBA-X                   597001 non-null  float64
 6   DBA-Y                   597001 non-null  float64
 7   DBA-Z                   597001 non-null  float64
 8   Pow-X                   597001 non-null  float64
 9   Pow-Y                   597001 non-null  float64
 10  Pow-Z                   597001 non-null  float64
 11  time_in_seconds         597001 non-null  float64
 12  Video_seconds           597001 non-null  float64
 13  Estatica-X              597001 non-null  float64
 14  Estatica-Y          